In [1]:
%display latex

Define the manifold and coordinates $(t, r, \theta, \phi)$

In [2]:
M = Manifold(4, 'M', structure='Lorentzian')
X.<t, r, th, ph> = M.chart(r't r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi')
X

Chart (M, (t, r, th, ph))

Introduce the variables that will be used.

In [3]:
s = var('s')
q, Q, m = var('q, Q, m')
kap = var('kappa')
R0 = var('R_0')
u = function('u')(s)
assume(s>0)
assume(kap>0)
assume(R0>kap)
assume(q>0)
assume(Q>0)
assume(m>0)
Cr = sqrt(R_0^2 - kap^2)
s, R0, kap, Cr

(s, R_0, kappa, sqrt(R_0^2 - kappa^2))

Define the metric for flat space in spherical polar coordinates, compute the Christoffel symbols, and define the affine connection for the manifold.

In [4]:
g = M.metric('g')
g[0, 0], g[1, 1], g[2, 2], g[3, 3] = -1, 1, r^2, r^2*sin(th)^2
g[:]

[           -1             0             0             0]
[            0             1             0             0]
[            0             0           r^2             0]
[            0             0             0 r^2*sin(th)^2]

In [5]:
Gam = g.christoffel_symbols()
nab = M.affine_connection('nabla', r'\nabla')
nab[:] = Gam[:]
nab[:]

[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]],
 [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, -r, 0], [0, 0, 0, -r*sin(th)^2]],
 [[0, 0, 0, 0], [0, 0, 1/r, 0], [0, 1/r, 0, 0], [0, 0, 0, -cos(th)*sin(th)]],
 [[0, 0, 0, 0],
  [0, 0, 0, 1/r],
  [0, 0, 0, cos(th)/sin(th)],
  [0, 1/r, cos(th)/sin(th), 0]]]

Restrict to the equatorial region $\theta = \pi/2$.

In [6]:
g.apply_map(lambda x: x.subs({th:pi/2}))
g[:]

[ -1   0   0   0]
[  0   1   0   0]
[  0   0 r^2   0]
[  0   0   0 r^2]

Define the elements of the quasi-null parallel propagated tetrad $(\dot{z}, n, e_i)$. The tangent vector to the null geodesic is $\dot{z}$, while $e_1$ is the normalised unit vector in the $\hat{\theta}$ direction. See the sage script titled ``Analysis_UR_Lim_coulomb'' for details about deriving $\dot{z}$.

In [7]:
zdot = M.vector_field(name=r'\dot{z}')
n = M.vector_field(name='n')
e1 = M.vector_field(name='e_1')
e2 = M.vector_field(name='e_2')
zdot[:] = [1/m, 1/m*sqrt(1 - kap^2/r^2), 0, kap/(m*r^2)]
e1[:] = [0, 0, 1/r, 0]
zdot[:], e1[:]

([1/m, sqrt(-kappa^2/r^2 + 1)/m, 0, kappa/(m*r^2)], [0, 0, 1/r, 0])

Check that the inner products between $\dot{z}$ and $e_1$ are satisfied, as well the fact that they are parallel transported along the null geodesic.

In [8]:
g(zdot, zdot).display(), g(zdot, e1).display(), g(e1, e1).display()

(g(\dot{z},\dot{z}): M → ℝ
    (t, r, th, ph) ↦ 0,
 g(\dot{z},e_1): M → ℝ
    (t, r, th, ph) ↦ 0,
 g(e_1,e_1): M → ℝ
    (t, r, th, ph) ↦ 1)

To check parallel transport, we need to substitute $\theta = \pi / 2$ in the affine connection.

In [9]:
Dzdot = nab(zdot)
De1 = nab(e1)
Dzdot.apply_map(lambda x: x.subs({th:pi/2}))
De1.apply_map(lambda x: x.subs({th:pi/2}))
Dzdot.contract(zdot)[:], De1.contract(zdot)[:]

([0, 0, 0, 0], [0, 0, 0, 0])

Define the variables $a_n,b_n, c_n$ that are the components of the vector $n^a$, and similarly for the components of $e_2$. Using the inner product relations, generate a set of equations that the components must satisfy.

In [10]:
an, bn, cn, a2, b2, c2 = var('a_n, b_n, c_n, a_2, b_2, c_2')
n[:] = [an, bn, 0, cn]
e2[:] = [a2, b2, 0, c2]
n[:], e2[:]

([a_n, b_n, 0, c_n], [a_2, b_2, 0, c_2])

In [11]:
g(n, zdot).display(), g(n, n).display(), g(n, e2).display()

(g(n,\dot{z}): M → ℝ
    (t, r, th, ph) ↦ (b_n*sqrt(kappa + r)*sqrt(-kappa + r) + (c_n*kappa - a_n)*r)/(m*r),
 g(n,n): M → ℝ
    (t, r, th, ph) ↦ c_n^2*r^2 - a_n^2 + b_n^2,
 g(n,e_2): M → ℝ
    (t, r, th, ph) ↦ c_2*c_n*r^2 - a_2*a_n + b_2*b_n)

In [12]:
g(e2, zdot).display(), g(e2, e2).display()

(g(e_2,\dot{z}): M → ℝ
    (t, r, th, ph) ↦ (b_2*sqrt(kappa + r)*sqrt(-kappa + r) + (c_2*kappa - a_2)*r)/(m*r),
 g(e_2,e_2): M → ℝ
    (t, r, th, ph) ↦ c_2^2*r^2 - a_2^2 + b_2^2)

In [13]:
assume(kap<r)
eq1 = (bn*sqrt(1 - kap^2/r^2) + (cn*kap - an))/(m) == -1
eq2 = cn^2*r^2 - an^2 + bn^2 == 0
eq3 = c2*cn*r^2 - a2*an + b2*bn == 0
eq4 = b2*sqrt(1 - kap^2/r^2) + c2*kap - a2 == 0
eq5 = c2^2*r^2 - a2^2 + b2^2 == 1
eq1

(c_n*kappa + b_n*sqrt(-kappa^2/r^2 + 1) - a_n)/m == -1

In [14]:
solve([eq1, eq2, eq3, eq4, eq5], an, bn, cn, a2, b2, c2)

[[a_n == 1/2*(r1^2 + 1)*m, b_n == 1/2*((r1^2 - 1)*m*r*sqrt(-(kappa^2 - r^2)/r^2) + 2*kappa*m*r1)/r, c_n == -1/2*(2*m*r*r1*sqrt(-(kappa^2 - r^2)/r^2) - (r1^2 - 1)*kappa*m)/r^2, a_2 == r1, b_2 == (r*r1*sqrt(-(kappa^2 - r^2)/r^2) + kappa)/r, c_2 == (kappa*r1 - r*sqrt(-(kappa^2 - r^2)/r^2))/r^2], [a_n == 1/2*(r2^2 + 1)*m, b_n == 1/2*((r2^2 - 1)*m*r*sqrt(-(kappa^2 - r^2)/r^2) - 2*kappa*m*r2)/r, c_n == 1/2*(2*m*r*r2*sqrt(-(kappa^2 - r^2)/r^2) + (r2^2 - 1)*kappa*m)/r^2, a_2 == r2, b_2 == (r*r2*sqrt(-(kappa^2 - r^2)/r^2) - kappa)/r, c_2 == (kappa*r2 + r*sqrt(-(kappa^2 - r^2)/r^2))/r^2], [a_n == 1/2*m, b_n == -1/2*m*sqrt(-(kappa^2 - r^2)/r^2), c_n == -1/2*kappa*m/r^2, a_2 == 0, b_2 == kappa/r, c_2 == -sqrt(-(kappa^2 - r^2)/r^2)/r], [a_n == 1/2*m, b_n == -1/2*m*sqrt(-(kappa^2 - r^2)/r^2), c_n == -1/2*kappa*m/r^2, a_2 == 0, b_2 == -kappa/r, c_2 == sqrt(-(kappa^2 - r^2)/r^2)/r]]

Solving the algebraic system generated by the different inner products, the solutions above are in terms of an unknown quantity $r_k, k \in \mathbb{Z}$. To resolve this, we need to impose the parallel transport constraint. Note that $r_k$ could in principle depend on the radial coordinate $r$. To avoid confusion, we denote this by the function $p(r)$. Furthermore, we will see that only the last solution is valid, i.e. $a_n = m/2, a_2 = 0$ and so on.

In [23]:
p = function('p')(r)
n[:] = [m/2*(p^2 + 1), ((p^2 - 1)*m*r*sqrt(1 - kap^2/r^2) - 2*kap*m*p)/(2*r), 0, (2*m*r*p*sqrt(1 - kap^2/r^2) - kap*m*(p^2 - 1))/(2*r^2)]
e2[:] = [p, (r*p*sqrt(1 - kap^2/r^2) - kap)/r, 0, (kap*p + r*sqrt(1 - kap^2/r^2))/r^2]
n[:], e2[:]

([1/2*(p(r)^2 + 1)*m,
  1/2*((p(r)^2 - 1)*m*r*sqrt(-kappa^2/r^2 + 1) - 2*kappa*m*p(r))/r,
  0,
  1/2*(2*m*r*sqrt(-kappa^2/r^2 + 1)*p(r) - (p(r)^2 - 1)*kappa*m)/r^2],
 [p(r),
  (r*sqrt(-kappa^2/r^2 + 1)*p(r) - kappa)/r,
  0,
  (kappa*p(r) + r*sqrt(-kappa^2/r^2 + 1))/r^2])

In [24]:
g(n, n).display(), g(n, zdot).display(), g(n, e2).display()

(g(n,n): M → ℝ
    (t, r, th, ph) ↦ -2*(kappa*m^2*p(r)^3 - kappa*m^2*p(r))*sqrt(kappa + r)*sqrt(-kappa + r)/r^2,
 g(n,\dot{z}): M → ℝ
    (t, r, th, ph) ↦ -(kappa^2*p(r)^2 - kappa^2 + r^2)/r^2,
 g(n,e_2): M → ℝ
    (t, r, th, ph) ↦ -(kappa^2*m*p(r)^3 - kappa^2*m*p(r) + (kappa*m*p(r)^2 - kappa*m)*sqrt(kappa + r)*sqrt(-kappa + r))/r^2)

In [25]:
g(e2, e2).display(), g(e2, zdot).display()

(g(e_2,e_2): M → ℝ
    (t, r, th, ph) ↦ 1,
 g(e_2,\dot{z}): M → ℝ
    (t, r, th, ph) ↦ 0)

Clearly $g(e_2, n) = 0$ only if $p(r) = 0$. Also check this satisfies parallel transport.

In [26]:
Dn = nab(n)
De2 = nab(e2)
acc_n = Dn.contract(zdot)
acc_e2 = De2.contract(zdot)
acc_n.apply_map(lambda x: x.subs({th:pi/2}))
acc_e2.apply_map(lambda x: x.subs({th:pi/2}))
acc_n[:], acc_e2[:]

([sqrt(kappa + r)*sqrt(-kappa + r)*p(r)*d(p)/dr/r,
  ((kappa^2*p(r)^2 - (kappa^2*r - r^3)*p(r)*d(p)/dr - kappa^2)*sqrt(kappa + r)*sqrt(-kappa + r) + (kappa^3*r - kappa*r^3)*d(p)/dr)/(sqrt(kappa + r)*sqrt(-kappa + r)*r^3),
  0,
  (kappa^3 - kappa*r^2 - (kappa^3 - kappa*r^2)*p(r)^2 - (kappa^2*r - r^3)*sqrt(kappa + r)*sqrt(-kappa + r)*d(p)/dr + (kappa^3*r - kappa*r^3)*p(r)*d(p)/dr)/(sqrt(kappa + r)*sqrt(-kappa + r)*r^4)],
 [sqrt(kappa + r)*sqrt(-kappa + r)*d(p)/dr/(m*r),
  -(kappa^2*r - r^3)*d(p)/dr/(m*r^3),
  0,
  -(kappa^3 - kappa*r^2)*d(p)/dr/(sqrt(kappa + r)*sqrt(-kappa + r)*m*r^3)])

The parallel transport condition is met only for $p(r) = constant$. Furthermore, $g(n, n) = 0$ implies $p = 0, \pm 1$. Choose $p=0$.

In [27]:
n[:] = [m/2, -m/2*sqrt(1 - kap^2/r^2), 0, -kap*m/(2*r^2)]
e2[:] = [0, -kap/r, 0, 1/r*sqrt(1 - kap^2/r^2)]
n[:], e2[:]

([1/2*m, -1/2*m*sqrt(-kappa^2/r^2 + 1), 0, -1/2*kappa*m/r^2],
 [0, -kappa/r, 0, sqrt(-kappa^2/r^2 + 1)/r])

In [28]:
g(n, n).display(), g(n, zdot).display(), g(n, e2).display()

(g(n,n): M → ℝ
    (t, r, th, ph) ↦ 0,
 g(n,\dot{z}): M → ℝ
    (t, r, th, ph) ↦ -1,
 g(n,e_2): M → ℝ
    (t, r, th, ph) ↦ 0)

In [29]:
g(e2, e2).display(), g(e2, zdot).display()

(g(e_2,e_2): M → ℝ
    (t, r, th, ph) ↦ 1,
 g(e_2,\dot{z}): M → ℝ
    (t, r, th, ph) ↦ 0)

In [30]:
Dn = nab(n)
De2 = nab(e2)
Dn.apply_map(lambda x: x.subs({th:pi/2}))
De2.apply_map(lambda x: x.subs({th:pi/2}))
Dn.contract(zdot)[:], De2.contract(zdot)[:]

([0, 0, 0, 0], [0, 0, 0, 0])

Clearly $n$ and $e_2$ defined this way satisfy the required conditions, and with this, we know the exact form of the quasi-null tetrad along the reference null geodesic. Essentially,
\begin{align}
\dot{z}^a &= \frac{1}{m} \bigg ( 1, \sqrt{1 - \frac{\kappa^2}{r^2}}, 0, \frac{\kappa}{r^2} \bigg ) \\
n^a &= \frac{m}{2} \bigg ( 1, - \sqrt{1 - \frac{\kappa^2}{r^2}}, 0, -\frac{\kappa}{r^2} \bigg ) \\
e^a_1 &= \bigg ( 0, 0, \frac{1}{r}, 0 \bigg ) \\
e^a_2 &= \bigg ( 0, -\frac{\kappa}{r}, 0, \frac{1}{r} \sqrt{1 - \frac{\kappa^2}{r^2}} \bigg )
\end{align}